In [1]:
#Injection Data API call
import requests
import csv
import json
from io import StringIO

# Set up our target URL
apiUrl = "https://injection.texnet.beg.utexas.edu/api/Export"

# Compose the export args dictionary
exportArgs = {
    "Format": "excel",
    "BeginYear": "2017",
    "BeginMonth": "01",
    "EndYear": "2024",
    "EndMonth": "01",
    # Include additional parameters as needed
}

headers = {
    "Content-type": "application/json",
    # Adjust based on actual response format:
    "Accept": "text/csv"
}

# Make the POST request
response = requests.post(apiUrl, headers=headers, data=json.dumps(exportArgs))

# Check if the request was successful
if response.status_code == 200:
    # Assuming the response is CSV, parse it
    csv_data = StringIO(response.text)
    csv_reader = csv.DictReader(csv_data)
    
    # Convert CSV to a list of dictionaries (similar to JSON)
    data_list = [row for row in csv_reader]
    
    # Print or process the data list as needed
    for row in data_list[:10]:  # Print only the first 10 rows
        print(row)
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")



{'UIC Number': '000114289', 'API Number': '31741196', 'Surface Longitude': '-101.69384147', 'Surface Latitude': '32.45895362', 'County Name': 'MARTIN', 'Field Name': '', 'Lease Name': 'TUBB  J. B. A/C-1', 'Well Number': '1', 'RRC District': '08', 'Operator Name': '', 'Operator Number': '326423', 'UIC Type': '1', 'Permit Number': '', 'Injection Top Interval': '10650', 'Injection Bottom Interval': '13500', 'MSIP': '', 'Well Permitted Formations': 'CAMBRIAN|DEVONIAN|ORDOVICIAN|SILURIAN|CAMBRIAN|DEVONIAN|ORDOVICIAN|SILURIAN| DEVONIAN| SILURIAN| CAMBRIAN| ORDOVICIAN', 'Injection Date': '2017-01-01', 'Injection End Date': '2017-01-01', 'Injection Pressure Average PSIG': '0', 'Injection Pressure Max PSIG': '0', 'Volume Injected (BBLs)': '0', 'Total Gas Volume (MCF)': '0', 'Annulus Pressure Num Readings': '', 'Annulus Pressure Min PSIG': '', 'Annulus Pressure Max PSIG': '', 'Current Injection Interval Top': '0', 'Current Injection Interval Bottom': '0', 'Current Injection Formations': '', 'Dep

In [2]:
#Save Injection Data in a Geojson file

import json
from io import StringIO
import requests
import csv

# Set up our target URL
apiUrl = "https://injection.texnet.beg.utexas.edu/api/Export"

# Compose the export args dictionary
exportArgs = {
    "Format": "excel",
    "BeginYear": "2017",
    "BeginMonth": "01",
    "EndYear": "2024",
    "EndMonth": "01",
}

headers = {
    "Content-type": "application/json",
    "Accept": "text/csv"  # Ensure the server supports and responds with CSV format
}

# Make the POST request
response = requests.post(apiUrl, headers=headers, data=json.dumps(exportArgs), verify=False)  # verify=False for demonstration; consider security implications

# Check if the request was successful
if response.status_code == 200:
    csv_data = StringIO(response.text)
    csv_reader = csv.DictReader(csv_data)
    
    # Initialize an empty GeoJSON structure
    geojson = {
        "type": "FeatureCollection",
        "features": []
    }
    
    # Convert each row in the CSV to a GeoJSON feature and add it to the GeoJSON structure
    for row in csv_reader:
        try:
            feature = {
                "type": "Feature",
                "geometry": {
                    "type": "Point",
                    "coordinates": [
                        float(row['Surface Longitude']),
                        float(row['Surface Latitude'])
                    ]
                },
                "properties": row  # All CSV columns become properties
            }
            geojson['features'].append(feature)
        except ValueError:
            # Handle rows with missing or invalid data
            continue
    # Write the GeoJSON data to a file
    with open('../data/injection_data.geojson', 'w') as f:
        json.dump(geojson, f, indent=4)

    print("GeoJSON data has been saved to 'data.geojson'.")
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")

/Users/brittanysmacbookair/anaconda3/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'injection.texnet.beg.utexas.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


GeoJSON data has been saved to 'data.geojson'.


In [1]:
import json
import requests
import csv
from io import StringIO

# Function to convert CSV response to GeoJSON
def csv_to_geojson(csv_content):
    csv_reader = csv.DictReader(StringIO(csv_content))
    features = []

    for row in csv_reader:
        try:
            # Extract and convert the required data from CSV
            properties = {
                "API Number": row.get("API Number", ""),
                "Volume Injected (BBLs)": float(row.get("Volume Injected (BBLs)", 0)),
                "County": row.get("County Name", ""),
                "Field Name": row.get("Field Name", ""),
                "Injection Date": row.get("Injection Date", ""),
                "Injection End Date": row.get("Injection End Date", ""),
                # Assuming 'Injection Layer' is related to 'Well Permitted Formations'
                "Injection Layer": row.get("Well Permitted Formations", "")
            }
            geometry = {
                "type": "Point",
                "coordinates": [
                    float(row.get("Surface Longitude", 0)),
                    float(row.get("Surface Latitude", 0))
                ]
            }

            features.append({
                "type": "Feature",
                "properties": properties,
                "geometry": geometry
            })
        except ValueError as e:
            print(f"Skipping row due to error: {e}")

    return {
        "type": "FeatureCollection",
        "features": features
    }

# Set up our target URL and export arguments
apiUrl = "https://injection.texnet.beg.utexas.edu/api/Export"
exportArgs = {
    "Format": "excel",
    "BeginYear": "2017",
    "BeginMonth": "01",
    "EndYear": "2024",
    "EndMonth": "12"
}
headers = {
    "Content-type": "application/json",
    "Accept": "text/csv"
}

# Make the POST request
response = requests.post(apiUrl, headers=headers, data=json.dumps(exportArgs), verify=False)

if response.status_code == 200:
    # Convert CSV to GeoJSON
    geojson_data = csv_to_geojson(response.text)
    
    # Write the GeoJSON data to a file in the 'data' folder
    with open('../data/injection_data_v2.geojson', 'w') as f:
        json.dump(geojson_data, f, indent=4)
    
    print("GeoJSON data has been saved to '../data/injection_data.geojson'.")
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")

/Users/brittanysmacbookair/anaconda3/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'injection.texnet.beg.utexas.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Skipping row due to error: could not convert string to float: ''
Skipping row due to error: could not convert string to float: ''
Skipping row due to error: could not convert string to float: ''
Skipping row due to error: could not convert string to float: ''
Skipping row due to error: could not convert string to float: ''
Skipping row due to error: could not convert string to float: ''
Skipping row due to error: could not convert string to float: ''
Skipping row due to error: could not convert string to float: ''
Skipping row due to error: could not convert string to float: ''
Skipping row due to error: could not convert string to float: ''
Skipping row due to error: could not convert string to float: ''
Skipping row due to error: could not convert string to float: ''
Skipping row due to error: could not convert string to float: ''
Skipping row due to error: could not convert string to float: ''
Skipping row due to error: could not convert string to float: ''
Skipping row due to error

In [2]:
import requests
import json

# Define the base URL for querying the Earthquake layer
query_url = "https://maps.texnet.beg.utexas.edu/arcgis/rest/services/catalog/catalog_all_flat/MapServer/0/query"

# Define the parameters for the query
params = {
    "where": "1=1",  # Fetches all earthquake records
    "outFields": "Magnitude,Latitude,Longitude,Event_Date,GeographicLocation",  # Specify the fields of interest
    "returnGeometry": True,  #might be redundant with f=geojson
    "outSR": "4326",  # Output Spatial Reference - WGS 84
    "f": "geojson"  # Request output format as GeoJSON
}

# Make the request to the ArcGIS REST API
response = requests.get(query_url, params=params)

# Check if the request was successful
if response.status_code == 200:
    # The response should already be in GeoJSON format
    geojson_data = response.json()

    # Save the GeoJSON data to a file
    with open('../data/earthquake_data.geojson', 'w') as f:
        json.dump(geojson_data, f, indent=4)

    print("Earthquake GeoJSON data has been saved to 'earthquake_data.geojson'.")
else:
    print(f"Failed to fetch data. Status code: {response.status_code}, Message: {response.text}")

Earthquake GeoJSON data has been saved to 'earthquake_data.geojson'.
